In [1]:
#Import necessary libraries

import numpy as np
import pandas as pd
import seal
from seal import *
import time
import pickle

In [2]:
models = pickle.load(open('models.dump','rb'))

In [3]:
#Model Owner initializes an encryption scheme

parms = EncryptionParameters(scheme_type.ckks)

#Model owner loads data owner's parameters

parms.load('encr_parms')

#MO initializes context with DO's parms

context = SEALContext(parms)

ckks_encoder = CKKSEncoder(context)
slot_count = ckks_encoder.slot_count()

keygen = KeyGenerator(context)
public_key = keygen.create_public_key()
public_key.load(context, 'pubkey')


galois_keys = keygen.create_galois_keys()
galois_keys.load(context, 'galkeys')


relin_keys = keygen.create_relin_keys()
relin_keys.load(context, 'relinkeys')

scale = pickle.load(open('scale','rb'))

encryptor = Encryptor(context, public_key)

In [4]:
def model_indices(models,slots):
    
    feat_list = []
    for model in models:
        batch = []
        index = []
        for item in model:
            batch.append(item[0]//slots)
            index.append(item[0]%slots)
            
        feat_list.append([batch, index])
        
    return feat_list

In [5]:
def models_to_vecs(models,slots):
    
    vec_list = []
    for model in models:
        vecs = []
        for item in model:
            empty = np.zeros(slots-1)
            vecs.append(np.insert(empty,item[0]%slots,item[1]))
            
        vec_list.append(vecs)
        
    return vec_list

In [6]:
start = time.time()

slots = 2048

model_data = models_to_vecs(models,slots)
#models_ct = model_data


for i in range(len(model_data)):
    for j in range(len(model_data[i])):
        vec_pt = ckks_encoder.encode(10**3 * model_data[i][j],scale)
        encryptor.encrypt(vec_pt).save('model_ct[%i][%i]'%(i,j))
        
end = time.time()
print(f"Time to encrypt and save model weights: {(end-start):.3f}s")

Time to encrypt and save model weights: 2.595s


In [7]:
pickle.dump(model_indices(models,2048), open('model_indices','wb'))